In [195]:
import argparse
import numpy as np
import pickle
import time
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime


from dataset import get_dataloader,EventData
from tqdm import tqdm
import random
from models.gated_tpp import *
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt

from models import sahp 


def load_data(name, dict_name):
    with open(name, 'rb') as f:
        data = pickle.load(f, encoding='latin-1')
        num_types = data['dim_process']
        data = data[dict_name]
        return data, int(num_types)

In [2]:
def power_law_kernel(t):  
    
    return 0.2 * (0.5 + t)**(-1.3)


class rational_quadratic_kernel(nn.Module):

    def __init__(self,
                 num_types, d_type, sigma=1, p=1, alpha=1,lengthscale = 1.0):
        super().__init__()

        self.d_type = d_type
        self.num_types = num_types
        self.norm = p
        # self.sigma = sigma
        self.param_loss = 0
        self.scores = None
        self.alpha =alpha
        """
        If the model is 1-D we still need the type embedding as an input and train a linear layer followed by softplus
        to make sure the length_scale parameter is positive. Adding a parameter followed by a sigmoid creates a non leaf
        tensor and there for doesn't work.
        """

        if num_types == 1:


            self.lengthscale = torch.nn.Parameter(torch.randn(1))
            self.base_intensity = 0.1
            self.sigma = torch.nn.Parameter(torch.randn(1))


    def forward(self, x):


        d = torch.abs(x[0] - x[1])**self.norm
        
        lengthscale = F.softplus(self.lengthscale,beta = 5)
        sigma = F.softplus(self.sigma,beta = 1)
        base_intensity = F.softplus(self.base_intensity,beta = 1)
        
        sigma= self.sigma
        lengthscale = self.lengthscale
        alpha = 0.6
#         self.base_intensity = 0.1

        self.scores = (sigma ** 2) * (1 + (d ** 2) / (alpha * lengthscale ** 2)) ** (-alpha)

        return self.scores
    def params(self, type_emb):

        params = []
        for space in type_emb[1:]:
            lengthscale = self.lengthscale(space).item()
            # alpha = self.alpha(space).item()
            alpha = self.alpha.item()

            params.append({'length_scale': lengthscale, 'alpha': alpha, 'sigma': self.sigma, 'Norm-P': self.norm})

        return params
    

In [1281]:
data = '../data/simulated/power_hawkes/'
batch_size = 1

kernel = rational_quadratic_kernel(1,1)

train_data, num_types = load_data(data + 'train.pkl', 'train')
dev_data, _ = load_data(data + 'dev.pkl', 'dev')
test_data, _ = load_data(data + 'test.pkl', 'test')
t_max = 1

trainloader = get_dataloader(train_data, 20, shuffle=False,t_max =t_max)
testloader = get_dataloader(test_data,40, shuffle=False,t_max = t_max)
valloader = get_dataloader(dev_data,40, shuffle=False,t_max =t_max)


valid_events = 0
test_events = 0
train_events = 0
for seq in valloader.dataset.event_type:
    valid_events += len(seq)
for seq in trainloader.dataset.event_type:
    train_events += len(seq)
for seq in testloader.dataset.event_type:
    test_events  += len(seq)

In [927]:


optimizer = optim.Adam(filter(lambda x: x.requires_grad, kernel.parameters()),
                      0.0001, betas=(0.9, 0.999), eps=1e-05, weight_decay=0.0)

optimizer.zero_grad()

losses = []
params = []
for epoch in range(100):
    epoch_loss= 0

    for batch in trainloader:
        event_time, arrival_time, event_type, _ = map(lambda x: x.to('cpu'), batch)

        lengths = (event_type != 0).sum(-1).to('cpu')
        xt_bar = event_time.unsqueeze(1). \
            expand(event_time.size(0), event_time.size(1), event_time.size(1))
        xt = xt_bar.transpose(1, 2)
        scores = kernel((xt_bar,xt))



        n_batch = xt.size()[0]
        length= xt.size()[1]
        samples = 1000

        ## Log Sum Intensities

        scores_0 = kernel((torch.ones(n_batch,length),torch.ones(n_batch,length)))
        intensities =scores 

        subsequent_mask=get_subsequent_mask(event_type)
        intensities = intensities.masked_fill_(subsequent_mask == 0, value=0)

        sample_intensities = intensities.sum(-1)-scores_0+kernel.base_intensity
        log_sum_intensities =nn.utils.rnn.pack_padded_sequence(sample_intensities.T, lengths - 1, enforce_sorted=False)[0].log().sum(-1)


        ## Non Event Intensities  MC Integration

        t_start = event_time[:,0]
        t_end =  event_time.max(-1)[0]
        constant_part = (sample_intensities[:,:-1]*arrival_time[:,1:]).sum(-1)


        mc_sample_values = (t_start.unsqueeze(-1)+torch.rand((n_batch,samples ))*(t_end -t_start).unsqueeze(-1))
        v = kernel((mc_sample_values,torch.zeros(mc_sample_values.size()))).mean(-1)

        non_event_intensity = (constant_part+functional_part +t_start*kernel.base_intensity).sum(-1)

        batch_loss = -(log_sum_intensities - non_event_intensity)

        epoch_loss+=batch_loss

        batch_loss.backward()
        optimizer.step()
        
    epoch_loss = (epoch_loss/train_events).item()
    print(f'Epoch:{epoch}, NLL:{epoch_loss:.6f}')
    

Epoch:0, NLL:2.603101
Epoch:1, NLL:2.593400
Epoch:2, NLL:2.580921
Epoch:3, NLL:2.566257
Epoch:4, NLL:2.549721
Epoch:5, NLL:2.531621
Epoch:6, NLL:2.512281
Epoch:7, NLL:2.492025
Epoch:8, NLL:2.471168
Epoch:9, NLL:2.450005
Epoch:10, NLL:2.428809
Epoch:11, NLL:2.407821
Epoch:12, NLL:2.387255
Epoch:13, NLL:2.367293
Epoch:14, NLL:2.348092
Epoch:15, NLL:2.329784
Epoch:16, NLL:2.312474
Epoch:17, NLL:2.296250
Epoch:18, NLL:2.281180
Epoch:19, NLL:2.267317
Epoch:20, NLL:2.254696
Epoch:21, NLL:2.243344
Epoch:22, NLL:2.233275
Epoch:23, NLL:2.224494
Epoch:24, NLL:2.216997
Epoch:25, NLL:2.210773
Epoch:26, NLL:2.205806
Epoch:27, NLL:2.202072
Epoch:28, NLL:2.199543
Epoch:29, NLL:2.198184
Epoch:30, NLL:2.197961
Epoch:31, NLL:2.198832
Epoch:32, NLL:2.200749
Epoch:33, NLL:2.203665
Epoch:34, NLL:2.207524
Epoch:35, NLL:2.212272
Epoch:36, NLL:2.217844
Epoch:37, NLL:2.224177
Epoch:38, NLL:2.231198
Epoch:39, NLL:2.238831
Epoch:40, NLL:2.246998
Epoch:41, NLL:2.255607
Epoch:42, NLL:2.264566
Epoch:43, NLL:2.27377

KeyboardInterrupt: 

In [ ]:
def function

In [928]:
F.softplus(kernel.sigma,beta = 1)

tensor([0.4511], grad_fn=<SoftplusBackward>)

In [929]:
F.softplus(kernel.lengthscale,beta = 1)

tensor([0.2789], grad_fn=<SoftplusBackward>)

## Validate the Loss Function

In [83]:


class rational_quadratic_kernel(nn.Module):

    def __init__(self,sigma=1,alpha=1,lengthscale = 1.0,base_intensity =0.2):
        super().__init__()



        """
        If the model is 1-D we still need the type embedding as an input and train a linear layer followed by softplus
        to make sure the length_scale parameter is positive. Adding a parameter followed by a sigmoid creates a non leaf
        tensor and there for doesn't work.
        """




        self.lengthscale = lengthscale
        self.base_intensity = base_intensity
        self.sigma = sigma
        self.alpha = alpha

    def forward(self, x):


        d = torch.abs(x[0] - x[1])
        self.scores = 0.2 * (0.5 +d)**(-1.3)

#         self.scores = (self.sigma ** 2) * (1 + (d ** 2) / (self.alpha * self.lengthscale ** 2)) ** (-self.alpha)

        return self.scores
    def params(self, type_emb):

        params = []
        for space in type_emb[1:]:
            lengthscale = self.lengthscale(space).item()
            # alpha = self.alpha(space).item()
            alpha = self.alpha.item()

            params.append({'length_scale': lengthscale, 'alpha': alpha, 'sigma': self.sigma, 'Norm-P': self.norm})

        return params

torch.manual_seed(42)
epoch_losses = []
sigmas = np.linspace(0.1,1.2,1)
for sigma in sigmas:
    kernel = rational_quadratic_kernel(sigma = 0.7, lengthscale=sigma,alpha = 0.6, base_intensity= 0.2)
    epoch_loss = 0
    for batch in trainloader:
        event_time, arrival_time, event_type, _ = map(lambda x: x.to('cpu'), batch)

        lengths = (event_type != 0).sum(-1).to('cpu')
        xt_bar = event_time.unsqueeze(1). \
            expand(event_time.size(0), event_time.size(1), event_time.size(1))
        xt = xt_bar.transpose(1, 2)
        scores = kernel((xt_bar,xt))



        n_batch = xt.size()[0]
        length= xt.size()[1]
#         samples = 1000

        ## Log Sum Intensities

        scores_0 = kernel((torch.ones(n_batch,length),torch.ones(n_batch,length)))
        intensities =scores 

        subsequent_mask=get_subsequent_mask(event_type)
        intensities = intensities.masked_fill_(subsequent_mask == 0, value=0)

        sample_intensities = intensities.sum(-1)-scores_0+kernel.base_intensity
        
#         sample_intensities = intensities.sum(-1)-scores_0+0.1

        

        log_sum_intensities =nn.utils.rnn.pack_padded_sequence(sample_intensities.T, lengths - 1, enforce_sorted=False)[0].log().sum(-1)


        ## Non Event Intensities  MC Integration

#         t_start = event_time[:,0]
#         t_end =  event_time.max(-1)[0]

#         constant_part = (sample_intensities[:,:-1]*arrival_time[:,1:]).sum(-1)


#         mc_sample_values = (t_start.unsqueeze(-1)+torch.rand((n_batch,samples ))*(t_end -t_start).unsqueeze(-1))
#         v = kernel((mc_sample_values,torch.zeros(mc_sample_values.size()))).mean(-1)

#         non_event_intensity = (constant_part+functional_part ).sum(-1)


        sample_arrival_time = arrival_time[:,1:]
        sample_event_time = event_time[:,:-1]
        n_mc_sample = 10
        n_batch = sample_arrival_time.size(0)
        n_t = sample_arrival_time.size(1)

        mc_values = torch.rand((n_batch,n_t,n_mc_sample ))*sample_arrival_time.unsqueeze(-1)+sample_event_time.unsqueeze(-1)

        xt_event= sample_event_time.unsqueeze(-1)*torch.ones((n_batch,n_t,n_mc_sample ))
        xt_bar = xt_event.unsqueeze(1).expand(xt_event.size(0), xt_event.size(1), xt_event.size(1), xt_event.size(2))
        xt = xt_bar.transpose(1, 2)

        xt_sample = mc_values
        xt_sample_bar = xt_sample.unsqueeze(1).expand(xt_sample.size(0), xt_sample.size(1), xt_sample.size(1), xt_sample.size(2))
        xt_sample = xt_sample_bar.transpose(1, 2)

        mc_mask = get_subsequent_mask(event_type[:,1:])

        non_event_intensity = kernel((xt_bar, xt_sample)).mean(-1)
        non_event_intensity = non_event_intensity.masked_fill_(mc_mask == 0, value=0)
        non_event_intensity = non_event_intensity.sum(-1)+kernel.base_intensity

        non_event_intensities =nn.utils.rnn.pack_padded_sequence(non_event_intensity.T, lengths - 1, enforce_sorted=False)[0].sum(-1)





        batch_loss = -(log_sum_intensities - non_event_intensities)

        epoch_loss+=batch_loss



    epoch_loss = (epoch_loss/train_events).item()

    epoch_losses.append(epoch_loss)

In [1273]:
non_event_intensities

tensor(18.3623)

In [1274]:
epoch_losses

[1.418138861656189]

In [1262]:
epoch_losses

[1.7872405052185059]

In [1275]:
epoch_losses

[1.418138861656189]

In [965]:
def kernel_fun(t):
    return (kernel.sigma ** 2) * (1 + (t ** 2) / (kernel.alpha * kernel.lengthscale ** 2)) ** (-kernel.alpha)

In [970]:
        ## Non Event Intensities  MC Integration

        t_start = event_time[:,0]
        t_end =  event_time.max(-1)[0]

        constant_part = (sample_intensities[:,:-1]*arrival_time[:,1:]).sum(-1)


        mc_sample_values = (t_start.unsqueeze(-1)+torch.rand((n_batch,samples ))*(t_end -t_start).unsqueeze(-1))
        v = kernel((mc_sample_values,torch.zeros(mc_sample_values.size()))).mean(-1)

        non_event_intensity = (constant_part+functional_part ).sum(-1)

        batch_loss = -(log_sum_intensities - non_event_intensity)

        epoch_loss+=batch_loss

0.1381845464077335

In [1077]:
kernel.sigma

0.7

In [1202]:
sample_arrival_time = arrival_time[:,1:]
sample_event_time = event_time[:,:-1]
n_mc_sample = 10
n_batch = sample_arrival_time.size(0)
n_t = sample_arrival_time.size(1)

mc_values = torch.rand((n_batch,n_t,n_mc_sample ))*sample_arrival_time.unsqueeze(-1)+sample_event_time.unsqueeze(-1)

xt_event= sample_event_time.unsqueeze(-1)*torch.ones((n_batch,n_t,n_mc_sample ))
xt_bar = xt_event.unsqueeze(1).expand(xt_event.size(0), xt_event.size(1), xt_event.size(1), xt_event.size(2))
xt = xt_bar.transpose(1, 2)

xt_sample = mc_values
xt_sample_bar = xt_sample.unsqueeze(1).expand(xt_sample.size(0), xt_sample.size(1), xt_sample.size(1), xt_sample.size(2))
xt_sample = xt_sample_bar.transpose(1, 2)

mc_mask = get_subsequent_mask(event_type[:,1:])

non_event_intensity = kernel((xt_bar, xt_sample)).mean(-1)
non_event_intensity = non_event_intensity.masked_fill_(mc_mask == 0, value=0)
non_event_intensity = non_event_intensity.sum(-1)+kernel.base_intensity

non_event_intensities =nn.utils.rnn.pack_padded_sequence(non_event_intensity.T, lengths - 1, enforce_sorted=False)[0].sum(-1)


In [1193]:
non_event_intensities

tensor(81.1080)

In [1200]:
non_event_intensity[0][:98]

tensor([0.1517, 0.1335, 0.2046, 0.5943, 0.1477, 0.5201, 0.1689, 0.2538, 0.4648,
        0.3040, 0.2520, 0.2212, 0.2082, 0.1657, 0.1333, 0.3494, 0.2570, 0.2208,
        0.2709, 0.5591, 0.4598, 0.1407, 0.1754, 0.2374, 0.3045, 0.2065, 0.1368,
        0.1416, 0.1299, 0.1510, 0.5377, 0.1482, 0.2143, 0.2060, 0.2244, 0.1943,
        0.1834, 0.1345, 0.1397, 0.3541, 0.2485, 0.1854, 0.1658, 0.1808, 0.2869,
        0.2839, 0.2033, 0.5245, 0.5232, 0.1896, 0.1693, 0.2772, 0.5179, 0.3203,
        0.2159, 0.2372, 0.2343, 0.3975, 0.2719, 0.3615, 0.2535, 0.4384, 0.3177,
        0.2466, 0.5769, 0.2786, 0.5765, 0.4167, 0.2001, 0.1342, 0.2018, 0.5632,
        0.2606, 0.4913, 0.6287, 0.3222, 0.6497, 0.5851, 0.7569, 0.2629, 0.2645,
        0.2704, 0.2155, 0.2781, 0.4020, 0.4314, 0.2010, 0.4162, 0.6847, 1.0569,
        0.8429, 0.9096, 0.6376, 0.3916, 0.6885, 0.2481, 0.5883, 0.2677])

In [1152]:
kernel_fun(21.6880- 21.1532) 

0.258316616710772

In [1197]:
length

99

In [1191]:
event_time.shape

torch.Size([5, 99])

In [1065]:
kernel_fun(6.328099999999999)+0.1

0.11711388426216773

In [1110]:
21.6880 -1.9428

19.745199999999997

In [1061]:
event_time

tensor([[[  8.2709],
         [ 20.9551],
         [ 21.6880],
         [ 30.6744],
         [ 33.2481],
         [ 34.3715],
         [ 34.5948],
         [ 47.2411],
         [ 55.4900],
         [ 56.2789],
         [ 57.7469],
         [ 63.2879],
         [ 67.9797],
         [ 68.6348],
         [ 70.2827],
         [ 72.5266],
         [ 77.2246],
         [ 84.5279],
         [ 91.0509],
         [ 92.9189],
         [ 92.9970],
         [ 93.2972],
         [ 95.5436],
         [ 99.8411],
         [109.1335],
         [109.7105],
         [115.0233],
         [123.4831],
         [133.3540],
         [150.4730],
         [156.3795],
         [162.2316],
         [168.2349],
         [168.3637],
         [170.9719],
         [184.7617],
         [188.2206],
         [190.9484],
         [194.9506],
         [196.1874]]])

In [1042]:
sample_event_time

tensor([[  1.9428,  17.4422,  21.1532,  21.9291,  33.2375,  33.4575,  34.4283,
          34.7124,  53.4867,  56.0763,  56.9533,  61.1035,  67.6659,  68.6015,
          68.9651,  71.2981,  76.6498,  77.9613,  85.5929,  92.5619,  92.9643,
          93.0787,  93.9720,  98.9322, 108.3831, 109.2280, 109.9768, 120.7993,
         133.1943, 142.5763, 156.1926, 156.6497, 168.1940, 168.2588, 168.3697,
         178.2014, 185.2074, 188.7799, 192.2796, 196.0970]])

In [959]:
sample_intensities[:,:-1]*arrival_time[:,1:]

tensor([[1.5499e+00, 3.9282e-01, 1.0618e-01, 3.5778e+00, 2.7426e-02, 5.2278e-01,
         1.1106e-01, 1.3293e+01, 3.2679e-01, 1.5128e-01, 1.3376e+00, 1.2020e+00,
         1.4517e-01, 1.1083e-01, 1.3833e+00, 1.4800e+00, 2.4672e-01, 2.1615e+00,
         1.1689e+00, 6.3755e-02, 5.4035e-02, 7.9269e-01, 2.7650e+00, 2.0513e+00,
         1.3761e-01, 2.4785e-01, 4.7237e+00, 1.9986e+00, 1.3526e+00, 1.9496e+00,
         6.1524e-02, 4.8983e+00, 9.1617e-03, 6.9108e-02, 1.0288e+01, 1.1118e+00,
         5.5501e-01, 6.2436e-01, 7.1769e-01, 4.0144e-01]])

In [953]:
non_event_intensity

tensor(2566.7708)

In [945]:
sigmas

array([0.05 , 0.125, 0.2  , 0.275, 0.35 , 0.425, 0.5  ])

## Learning Params for Exponential Kernel

In [4]:
data = '../data/simulated/power_hawkes/'
batch_size = 1

# kernel = rational_quadratic_kernel(1,1)

train_data, num_types = load_data(data + 'train.pkl', 'train')
dev_data, _ = load_data(data + 'dev.pkl', 'dev')
test_data, _ = load_data(data + 'test.pkl', 'test')
t_max = 1

trainloader = get_dataloader(train_data, 20, shuffle=False,t_max =t_max)
testloader = get_dataloader(test_data,40, shuffle=False,t_max = t_max)
valloader = get_dataloader(dev_data,40, shuffle=False,t_max =t_max)


valid_events = 0
test_events = 0
train_events = 0
for seq in valloader.dataset.event_type:
    valid_events += len(seq)
for seq in trainloader.dataset.event_type:
    train_events += len(seq)
for seq in testloader.dataset.event_type:
    test_events  += len(seq)

In [13]:


class exponential_kernel(nn.Module):

    def __init__(self,sigma=1,alpha=1,lengthscale = 1.0,base_intensity =0.2):
        super().__init__()



        """
        If the model is 1-D we still need the type embedding as an input and train a linear layer followed by softplus
        to make sure the length_scale parameter is positive. Adding a parameter followed by a sigmoid creates a non leaf
        tensor and there for doesn't work.
        """




        self.lengthscale = torch.nn.Parameter(torch.randn(1))
        self.base_intensity = torch.nn.Parameter(torch.randn(1))
        self.sigma = torch.nn.Parameter(torch.randn(1))

    

    def forward(self, x):
        
        
        base_intensity = F.softplus(self.base_intensity,beta = 10)
        lengthscale = F.softplus(self.lengthscale,beta = 5)
        sigma = F.softplus(self.sigma,beta =5)
        
        
        d = torch.abs(x[0] - x[1])
        self.scores = sigma*torch.exp(-d/lengthscale)
        
        

#         self.scores = (self.sigma ** 2) * (1 + (d ** 2) / (self.alpha * self.lengthscale ** 2)) ** (-self.alpha)

        return self.scores


torch.manual_seed(42)
epoch_losses = []


kernel = exponential_kernel(sigma = 0.6, lengthscale=0.7, base_intensity= 0.1)
optimizer = optim.Adam(filter(lambda x: x.requires_grad, kernel.parameters()),
                      0.0000, betas=(0.9, 0.999), eps=1e-05, weight_decay=0.0)

optimizer.zero_grad()
for epoch in range(100):
    epoch_loss = 0
    for batch in trainloader:
        event_time, arrival_time, event_type, _ = map(lambda x: x.to('cpu'), batch)

        lengths = (event_type != 0).sum(-1).to('cpu')
        xt_bar = event_time.unsqueeze(1). \
            expand(event_time.size(0), event_time.size(1), event_time.size(1))
        xt = xt_bar.transpose(1, 2)
        scores = kernel((xt_bar,xt))



        n_batch = xt.size()[0]
        length= xt.size()[1]
    #         samples = 1000
    
        sigma = F.softplus(kernel.sigma,beta = 1)
        base_intensity = F.softplus(kernel.base_intensity,beta = 10)
        lengthscale = F.softplus(kernel.lengthscale,beta = 5)

        ## Log Sum Intensities

        scores_0 = kernel((torch.ones(n_batch,length),torch.ones(n_batch,length)))
        intensities =scores 

        subsequent_mask=get_subsequent_mask(event_type)
        intensities = intensities.masked_fill_(subsequent_mask == 0, value=0)

        sample_intensities = intensities.sum(-1)-scores_0+base_intensity


        log_sum_intensities =nn.utils.rnn.pack_padded_sequence(sample_intensities.T, lengths - 1, enforce_sorted=False)[0].log().sum(-1)


        ## Non Event Intensities Integral

        t_start = event_time[:,0]
        t_end =  event_time.max(-1)[0]

        constant_part = (sample_intensities[:,:-1]*arrival_time[:,1:]).sum(-1)
        functional_part =(sigma*arrival_time[:,1:]*torch.exp(-arrival_time[:,1:]/lengthscale)).sum(-1)
        non_event_intensities = constant_part+functional_part +torch.abs((t_start - t_end)*base_intensity)


        batch_loss = -(log_sum_intensities - non_event_intensities.sum())
        epoch_loss+=batch_loss

        batch_loss.backward()
        optimizer.step()




    epoch_loss = (epoch_loss/train_events).item()
    print(f'Epoch:{epoch}, NLL:{epoch_loss:.6f}')
    print(f'Lengthscale:{lengthscale.item():.6f}')
    print(f'Sigma:{sigma.item():.6f}')
    print(f'Base_intensity:{base_intensity.item():.6f}\n')

    
    
    epoch_losses.append(epoch_loss)

C:\Users\yamac\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.


Epoch:0, NLL:2.818700
Lengthscale:0.380243
Sigma:0.823427
Base_intensity:0.144401

Epoch:1, NLL:2.802478
Lengthscale:0.392188
Sigma:0.830846
Base_intensity:0.134962

Epoch:2, NLL:2.788753
Lengthscale:0.404574
Sigma:0.838354
Base_intensity:0.126048



KeyboardInterrupt: 

## Learning from Sinusoid Kernel

In [3]:
data = '../data/simulated/sin_hawkes/'
batch_size = 1

# kernel = rational_quadratic_kernel(1,1)

train_data, num_types = load_data(data + 'train.pkl', 'train')
dev_data, _ = load_data(data + 'dev.pkl', 'dev')
test_data, _ = load_data(data + 'test.pkl', 'test')
t_max = 1

trainloader = get_dataloader(train_data, 20, shuffle=False,t_max =t_max)
testloader = get_dataloader(test_data,40, shuffle=False,t_max = t_max)
valloader = get_dataloader(dev_data,40, shuffle=False,t_max =t_max)


valid_events = 0
test_events = 0
train_events = 0
for seq in valloader.dataset.event_type:
    valid_events += len(seq)
for seq in trainloader.dataset.event_type:
    train_events += len(seq)
for seq in testloader.dataset.event_type:
    test_events  += len(seq)

In [4]:


class rayleigh_kernel(nn.Module):

    def __init__(self,sigma=1,alpha=1,lengthscale = 1.0,base_intensity =0.2):
        super().__init__()



        """
        If the model is 1-D we still need the type embedding as an input and train a linear layer followed by softplus
        to make sure the length_scale parameter is positive. Adding a parameter followed by a sigmoid creates a non leaf
        tensor and there for doesn't work.
        """



        self.omega = torch.nn.Parameter(torch.randn(1))
        self.base_intensity = torch.nn.Parameter(torch.randn(1))
        self.s = torch.nn.Parameter(torch.randn(1))

    

    def forward(self, x):
        
        
        base_intensity = F.softplus(self.base_intensity,beta = 10)
        s = F.softplus(self.s,beta = 0.7)
        omega = torch.sigmoid(self.omega)

        
        
        d = torch.abs(x[0] - x[1])
        self.scores = omega*d*torch.exp(-omega*d/s**2)
        
        

#         self.scores = (self.sigma ** 2) * (1 + (d ** 2) / (self.alpha * self.lengthscale ** 2)) ** (-self.alpha)

        return self.scores

    def integral(self,x):
        
        s = F.softplus(self.s,beta = 0.7)

        omega = torch.sigmoid(self.omega)

        a = (s**2)
        b = torch.exp(-(omega*s)/(x+1e-6)**2)
        c = (s**2+omega*x)
        return (a*b*c)/omega


torch.manual_seed(42)
epoch_losses = []


kernel = rayleigh_kernel(sigma = 0.6, lengthscale=0.7, base_intensity= 0.1)
optimizer = optim.Adam(filter(lambda x: x.requires_grad, kernel.parameters()),
                      0.00005, betas=(0.9, 0.999), eps=1e-05, weight_decay=0.0)

optimizer.zero_grad()
for epoch in range(100):
    epoch_loss = 0
    for batch in trainloader:
        event_time, arrival_time, event_type, _ = map(lambda x: x.to('cpu'), batch)

        lengths = (event_type != 0).sum(-1).to('cpu')
        xt_bar = event_time.unsqueeze(1). \
            expand(event_time.size(0), event_time.size(1), event_time.size(1))
        xt = xt_bar.transpose(1, 2)
        scores = kernel((xt_bar,xt))

        n_batch = xt.size()[0]
        length= xt.size()[1]


        base_intensity = F.softplus(kernel.base_intensity,beta = 10)
        s = F.softplus(kernel.s,beta = 0.7)
        omega = torch.sigmoid(kernel.omega)


        ## Log Sum Intensities

        scores_0 = kernel((torch.ones(n_batch,length),torch.ones(n_batch,length)))
        intensities =scores 

        subsequent_mask=get_subsequent_mask(event_type)
        intensities = intensities.masked_fill_(subsequent_mask == 0, value=0)
        sample_intensities = intensities.sum(-1)-scores_0+base_intensity
        log_sum_intensities =nn.utils.rnn.pack_padded_sequence(sample_intensities.T, lengths - 1, enforce_sorted=False)[0].log().sum(-1)


        ## Non Event Intensities Integral

        t_start = event_time[:,0]
        t_end =  event_time.max(-1)[0]


        constant_part = (sample_intensities[:,:-1]*arrival_time[:,1:]).sum(-1)

        functional_part =kernel.integral(arrival_time[:,1:]).sum(-1)
        non_event_intensities = constant_part+functional_part +torch.abs((t_start - t_end)*base_intensity)

        batch_loss = -(log_sum_intensities - non_event_intensities.sum())
        epoch_loss+=batch_loss

        batch_loss.backward()
        optimizer.step()




    epoch_loss = (epoch_loss/train_events).item()
    print(f'Epoch:{epoch}, NLL:{epoch_loss:.6f}')
    print(f'omega:{omega.item():.6f}')
    print(f's:{s.item():.6f}')
    print(f'Base_intensity:{base_intensity.item():.6f}\n')

    
    
    epoch_losses.append(epoch_loss)

Epoch:0, NLL:9.886042
omega:0.586136
s:1.106126
Base_intensity:0.144395

Epoch:1, NLL:9.684965
omega:0.589240
s:1.099218
Base_intensity:0.134765

Epoch:2, NLL:9.485313
omega:0.592320
s:1.092364
Base_intensity:0.125514



KeyboardInterrupt: 

In [35]:
x = arrival_time[:,1:]
omega = F.softplus(kernel.omega,beta =1)
a = (s**2)
b = torch.exp(-(omega*s)/x**2)

In [36]:
 b = -(omega*s)/x**2

In [37]:
b

tensor([[-1.2908e-01, -5.4327e-02, -9.5139e-02,  ..., -1.2311e-01,
         -6.7911e+01, -2.5823e-01],
        [-2.0101e+01, -4.6918e-03, -7.5552e-03,  ...,        -inf,
                -inf,        -inf],
        [-8.4493e-03, -1.6205e-03, -6.6116e-02,  ...,        -inf,
                -inf,        -inf],
        ...,
        [-1.7616e+00, -5.4808e-01, -4.5753e+02,  ...,        -inf,
                -inf,        -inf],
        [-1.1879e+00, -3.1272e-01, -4.6402e+02,  ...,        -inf,
                -inf,        -inf],
        [-2.9649e+01, -7.3931e+01, -1.3559e+00,  ...,        -inf,
                -inf,        -inf]], grad_fn=<DivBackward0>)

## Getting the log likelihood


In [112]:

def load_data_loaders(data_path = '../data/simulated/power_hawkes/',batch_size = 40):




    train_data, num_types = load_data(data_path + 'train.pkl', 'train')
    dev_data, _ = load_data(data_path + 'dev.pkl', 'dev')
    test_data, _ = load_data(data_path + 'test.pkl', 'test')

    trainloader = get_dataloader(train_data, batch_size, shuffle=False,t_max =1)
    testloader = get_dataloader(test_data,batch_size, shuffle=False,t_max = 1)
    valloader = get_dataloader(dev_data,batch_size, shuffle=False,t_max =1)


    valid_events = 0
    test_events = 0
    train_events = 0

    for seq in valloader.dataset.event_type:
        valid_events += len(seq)
    for seq in trainloader.dataset.event_type:
        train_events += len(seq)
    for seq in testloader.dataset.event_type:
        test_events  += len(seq)

    return (trainloader,valloader,testloader,train_events,valid_events,test_events)


class magic_kernel(nn.Module):

    def __init__(self,
                 num_types=1, d_type=1, sigma=1, p=1, alpha=1, lengthscale=1.0, betas=[5, 1, 1]):
        super().__init__()

        self.d_type = d_type
        self.num_types = num_types
        self.norm = p
        self.param_loss = 0
        self.scores = None
        self.sigma = sigma
        self.alpha = alpha
        self.lengthscale = lengthscale
        self.betas = betas

        """
        If the model is 1-D we still need the type embedding as an input and train a linear layer followed by softplus
        to make sure the length_scale parameter is positive. Adding a parameter followed by a sigmoid creates a non leaf
        tensor and there for doesn't work.
        """

#         if num_types == 1:

#             self.lengthscale = torch.nn.Parameter(torch.randn(1))
# #             self.base_intensity = torch.nn.Parameter(torch.randn(1))
#             self.base_intensity =torch.tensor(-2.3)
#             self.sigma = torch.nn.Parameter(torch.randn(1))
#             self.alpha = torch.nn.Parameter(torch.randn(1))

#         else:
#             self.lengthscale = nn.Sequential(nn.Linear(d_type * 2, 1, bias=False), nn.Softplus(beta = 10))
#             self.alpha = nn.Sequential(nn.Linear(d_type * 2, 1, bias=False), nn.Softplus(beta = 1))
#             self.sigma = nn.Sequential(nn.Linear(d_type * 2, 1, bias=False), nn.Sigmoid())
#             self.base_intensity = nn.Sequential(nn.Linear(d_type, 1, bias=False), nn.Softplus())

    def forward(self, time_diff, combined_embeddings=None,non_event_intensity = False)  :      

        lengthscale = self.lengthscale
        sigma = self.sigma
        alpha = self.alpha        
        d = time_diff

#         if self.num_types == 1:
# #             lengthscale = F.softplus(self.lengthscale)
# #             sigma = torch.sigmoid(self.sigma)      
# #             alpha = F.softplus(self.alpha)
# #             base_intensity = F.softplus(self.base_intensity)

#         else:
#             if not non_event_intensity:
#                 lengthscale = self.lengthscale(combined_embeddings).squeeze(-1)
#                 sigma = self.sigma(combined_embeddings).squeeze(-1)
#                 alpha = self.alpha(combined_embeddings).squeeze(-1)

#             else:
#                 lengthscale = self.lengthscale(combined_embeddings)
#                 sigma = self.sigma(combined_embeddings)
#                 alpha = self.alpha(combined_embeddings)

#             base_intensity = self.base_intensity(combined_embeddings[:, :, :, self.d_type:]).squeeze(-1)

        # (sigma ** 2) * (1 + (d ** 2) / (self.alpha * lengthscale ** 2)) ** (-self.alpha)
        #
        # (sigma ** 2) * (1 + (d ** 2) / (self.alpha * lengthscale ** 2)) ** (-self.alpha) *((1 + torch.exp(-d)) ** -alpha)
        self.scores = sigma * torch.exp(-d / lengthscale) * ((1 + torch.exp(-d)) ** -alpha)
        # self.scores = (sigma ** 2) * (1 + (d ** 2) / (1 * lengthscale ** 2)) ** (-1) *((1 + torch.exp(-d)) ** -alpha)

        return self.scores
    
    
def get_sample_intensities(kernel,event_time, arrival_time, event_type, device='cpu', embeddings=None,base_intensity=0.1):

    # event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)

    xt_bar, xt = get_pairwise_times(event_time)
    t_diff = torch.abs(xt_bar - xt)
    n_batch = t_diff.size()[0]
    length_batch = t_diff.size()[1]

    if kernel.num_types == 1:
        scores = kernel(t_diff)
        scores_0 = kernel(torch.zeros(n_batch, length_batch).to(device))  # We need this to get actual intensities
  
    else:

        pair_wise_embeddings = get_pairwise_type_embeddings(embeddings)
        combined_embeddings = torch.cat([pair_wise_embeddings[0], pair_wise_embeddings[1]],
                                        dim=-1)  # Last Part is the current event
        scores = kernel(t_diff, combined_embeddings)
        scores_0 = kernel(torch.zeros(n_batch, length_batch, length_batch).to(device),
                          combined_embeddings)  # We need this to get actual intensities
        scores_0 = torch.diagonal(scores_0, dim1=1, dim2=-1)

    subsequent_mask = get_subsequent_mask(event_type)
    sample_intensities = scores.masked_fill_(subsequent_mask == 0, value=0).sum(-1)

    sample_intensities = scores.sum(-1) - scores_0
    seq_length_mask = (event_type != 0) * 1

    return (sample_intensities + base_intensity) * seq_length_mask


def get_non_event_intensities(kernel,  event_time, arrival_time, event_type,
                              type_embeddings=None, device='cpu',mc_sample_size = 5,base_intensity =  0.1,base_intensities = None):

    # event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)

    sample_arrival_time = arrival_time[:, 1:]
    sample_event_time = event_time[:, :-1]
    t_last = sample_event_time.max(-1)[0]
    n_batch = sample_arrival_time.size(0)
    n_t = sample_arrival_time.size(1)

    mc_values = torch.rand((n_batch, n_t, mc_sample_size)).to(device) * \
                sample_arrival_time.unsqueeze(-1) + sample_event_time.unsqueeze(-1)

    samples = sample_event_time.unsqueeze(-1).expand((n_batch, n_t, mc_sample_size))
    samples = samples.unsqueeze(1).expand(samples.size(
        0), samples.size(1), samples.size(1), samples.size(-1))

    mc_values_bar = mc_values.unsqueeze(1).expand(mc_values.size(
        0), mc_values.size(1), mc_values.size(1), mc_values.size(-1))

    mc_values_bar = mc_values_bar.transpose(1, 2)
    d = torch.abs((mc_values_bar - samples))

    if kernel.num_types == 1:

        non_event_intensities = kernel(d)
        trigger_integral = non_event_intensities.mean(-1)
        subsequent_mask = get_subsequent_mask(event_type[:, 1:])
        integral = trigger_integral * subsequent_mask
        integral = integral.sum(-1)
        seq_length_mask = (event_type[:, 1:] != 0) * 1
        integral = integral * seq_length_mask
#         base_intensity = F.softplus(kernel.base_intensity, beta=1)
        base_intensity = base_intensity

        sequence_integral =integral.sum(-1)+base_intensity*t_last
    else:
        sequence_integral = 0
        for i in range(kernel.num_types):
            sample_event_types = event_type[:, 1:]
            sample_embeddings = type_embeddings(sample_event_types)
            xd_bar = sample_embeddings.unsqueeze(1).expand(sample_embeddings.size(
                0), sample_embeddings.size(1), sample_embeddings.size(1), sample_embeddings.size(-1))

            current_embedding = type_embeddings(torch.tensor([i]).to(device))
            current_embedding = current_embedding[:, None, None:].expand(xd_bar.size()).transpose(1, 2)
            combined_embeddings = torch.cat([xd_bar, current_embedding], dim=-1)

            non_event_intensities = kernel(d, combined_embeddings, non_event_intensity=True)
            trigger_integral = non_event_intensities.mean(-1)
            subsequent_mask = get_subsequent_mask(sample_event_types)
            integral = trigger_integral * subsequent_mask
            integral = integral.sum(-1)
            seq_length_mask = (event_type[:, 1:] != 0) * 1
            integral = integral * seq_length_mask
#             base_intensity = kernel.base_intensity(type_embeddings(torch.tensor([i]).to(device)))
            base_intensity = base_intensities[i]
            sequence_integral += integral.sum(-1) + base_intensity * t_last

    return sequence_integral






In [189]:
device = 'cuda'
num_types = 2
type_embeddings = nn.Embedding(num_types + 1, dtype, padding_idx=0).to(device)


In [109]:
torch.sigmoid(1)

TypeError: sigmoid(): argument 'input' (position 1) must be Tensor, not int

In [183]:
sigmas = [0.1,0.7,1.0]
lengthscales = [0.1,1.0,2.0]
alphas = []
base_intensities = np.linspace(0,1,10)



kernel = kernel_functions.magic_kernel(2,sigma=0.5, alpha=1.0 , lengthscale=1.3).to(device)

trainloader,valloader,testloader,train_events,valid_events,test_events = load_data_loaders(data_path = '../data/simulated/sin_hawkes/',batch_size = 40)
total_loss =0
for batch in trainloader:
    batch
#     event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)
#     sample_intensities = kernel_functions.get_sample_intensities(kernel,event_time, arrival_time, event_type,
#                                                      device=device, embeddings=type_embeddings,base_intensity = 0.2)
#     sample_intensities[sample_intensities==0] =1
#     sample_intensities = sample_intensities.log().sum(-1)
#     non_event_intensities = get_non_event_intensities(kernel,  event_time, arrival_time,
#                                                                    event_type,
#                           type_embeddings=type_embeddings, device=device,mc_sample_size = 5,base_intensity = 0.2)
#     nll_loss = -(sample_intensities-non_event_intensities).sum()
#     total_loss+=nll_loss


In [194]:
kernel.base_intensity(type_embeddings(torch.tensor(1).to(device)))

tensor([0.6990], device='cuda:0', grad_fn=<SoftplusBackward>)

In [193]:
type_embeddings(torch.tensor(1).to(device))

tensor([-0.0388,  0.8927, -2.1533,  1.2202,  0.2024,  0.4263,  0.3830,  0.8494,
        -1.4979,  1.0359, -0.2364, -1.7491,  0.6800, -0.1618, -0.6520, -0.4840,
        -0.3222, -0.6776,  0.1516,  1.1664, -0.0495,  0.5862, -1.9633, -1.0977,
        -0.2065, -0.7687,  0.6140, -0.6840,  1.0882,  0.8283, -1.1135, -0.5497],
       device='cuda:0', grad_fn=<EmbeddingBackward>)

In [149]:
total_loss

tensor(365150.4688, device='cuda:0')

In [151]:
total_loss

tensor(351932.0625, device='cuda:0')

In [153]:
total_loss

tensor(342955.5000, device='cuda:0')

In [101]:
kernel.base_intensity

tensor(-2.3000)

In [102]:
F.softplus(torch.tensor(-2.3), beta=kernel.betas[-1])

tensor(0.0955)

tensor(-2.3000)

In [51]:


sample_intensities = kernel_functions.get_sample_intensities(self.encoder.kernel,event_time, batch_arrival_times, batch_types,
                                                             device=device, embeddings=embeddings)
sample_intensities = sample_intensities.sum(-1)
non_event_intensities = kernel_functions.get_non_event_intensities(self.encoder.kernel,  event_time, batch_arrival_times,
                                                                   batch_types,
                      type_embeddings=self.encoder.type_emb, device=device,mc_sample_size = 5)
nll_loss = -(sample_intensities-non_event_intensities).sum()

NameError: name 'params' is not defined

In [44]:
event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)
embeddings = type_embeddings(event_type)
sample_intensities = kernel_functions.get_sample_intensities(kernel,batch,device= device,embeddings =embeddings)
non_event_intensities =get_non_event_intensities(kernel, batch,type_embeddings=type_embeddings, device='cpu',mc_sample_size = 5)

In [48]:
sample_intensities[0]

tensor([1.1426, 0.6352, 0.6384, 1.1430, 1.1426, 1.1557, 1.3196, 0.6404, 0.9150,
        0.6348, 0.8168, 1.1452, 1.4838, 1.9160, 1.6435, 1.2997, 1.2121, 0.9370,
        1.5060, 0.8490, 1.8299, 1.5522, 1.6921, 1.5630, 2.0960, 1.2370, 1.2769,
        1.6783, 1.7655, 0.8479, 1.1426, 0.8833, 2.0595, 1.9876, 2.3234, 2.7976,
        0.6348, 1.0404, 2.0586, 1.1432, 1.4328, 0.7026, 0.6850, 1.7039, 1.1441,
        1.2505, 1.1796, 1.1190, 1.6459, 2.0817, 0.7079, 0.6351, 0.6698, 0.7811,
        1.7351, 2.4722, 2.1004, 2.4733, 2.0668, 3.4276, 1.2506, 0.6412, 0.9228,
        1.2614, 0.6752, 1.1738, 0.8629, 2.0139, 2.5608, 1.4336, 2.5385, 2.1294,
        1.1210, 1.4485, 1.2259, 0.6360, 1.1172, 0.6778, 0.6365, 0.6602, 1.8127,
        1.2822, 1.9790, 0.9001, 1.1086, 1.5179, 1.2142, 2.3429, 1.3730, 1.9425,
        1.3102, 2.2970, 3.0335, 1.1996, 1.6552, 1.7434, 0.6350, 1.6496, 2.0652,
        1.6581, 1.1713, 1.2430, 0.7974, 0.6453, 1.1760, 1.7805, 2.2146, 0.7789,
        1.3991, 0.7759, 1.9977, 1.6216, 

In [45]:
non_event_intensities

tensor([[ 461.1487,  587.7064,  507.8156,  607.8556,  496.6086,  646.5068,
         1034.2786,  706.8341,  582.9006,  805.8822,  406.0864,  711.5400,
          608.1233,  451.5201,  771.9742,  502.5106,  588.0792,  518.2933,
          379.4916,  570.9720,  618.3298,  586.9587,  462.9715,  663.7072,
          482.9331,  500.2524,  805.8136,  540.2129,  583.9819,  597.1379,
          425.7408,  633.5850,  585.1597,  521.8990,  747.3402,  638.8920,
          488.0743,  451.7670,  484.7585,  604.8957]], grad_fn=<AddBackward0>)

In [8]:
event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)
mc_sample_size = 2
sample_arrival_time = arrival_time[:,1:]
sample_event_time = event_time[:,:-1]
t_last = sample_event_time.max(-1)[0]
n_batch = sample_arrival_time.size(0)
n_t = sample_arrival_time.size(1)


In [9]:
mc_values = torch.rand((n_batch,n_t,mc_sample_size)).to(device)*sample_arrival_time.unsqueeze(-1) +event_time[:,:-1].unsqueeze(-1)

In [10]:
samples = sample_event_time.unsqueeze(-1).expand((n_batch,n_t,mc_sample_size))
samples = samples.unsqueeze(1).expand(samples.size(
        0), samples.size(1), samples.size(1), samples.size(-1))

In [11]:
mc_values_bar = mc_values.unsqueeze(1).expand(mc_values.size(
        0), mc_values.size(1), mc_values.size(1), mc_values.size(-1))

mc_values_bar = mc_values_bar.transpose(1, 2)

In [12]:
d = torch.abs((mc_values_bar - samples))

# non_event_intensities = kernel(d)
# trigger_integrals = non_event_intensities.mean(-1)
# subsequent_mask = get_subsequent_mask(event_type[:,1:])
# integral = trigger_integral*subsequent_mask
# integral = integral.sum(-1)
# seq_length_mask = (event_type[:,1:] != 0) * 1
# integral = integral*seq_length_mask

# base_intensity = F.softplus(kernel.base_intensity, beta=1)

# sequence_integral =integral.sum(-1)+base_intensity*t_last

In [19]:
event_time[:-1].shape

torch.Size([39, 331])

In [20]:
sample_arrival_time = arrival_time[:, 1:]
sample_event_time = event_time[:, :-1]
t_last = sample_event_time.max(-1)[0]
n_batch = sample_arrival_time.size(0)

n_t = sample_arrival_time.size(1)

mc_values = torch.rand((n_batch, n_t, 2)).to(device) * \
            sample_arrival_time.unsqueeze(-1) +sample_event_time.unsqueeze(-1)

In [13]:
sample_types = type_embeddings(torch.tensor([range(1,num_types+1)]))

In [28]:
compensator =get_non_event_intensities(kernel, batch,type_embeddings=type_embeddings, device='cpu',mc_sample_size = 5)

In [29]:
compensator

tensor([ 99.6408,  88.0461,  96.3559,  98.2445,  98.3215,  85.4530, 105.7602,
         94.5943,  91.9119,  99.1646,  92.3523,  95.8380,  93.0733,  94.8377,
         92.8882,  86.0339,  88.0227,  91.5060,  96.5376,  96.7593,  96.2483,
         91.1681,  89.7881,  93.1762,  97.8715,  91.2969,  91.7242,  89.0858,
         98.3729,  99.1456,  96.3078,  95.1227,  94.7207,  92.0839,  88.7675,
         97.9171,  95.1148,  86.1241,  93.4037,  94.5680],
       grad_fn=<AddBackward0>)

In [23]:
def get_non_event_intensities(kernel, batch,type_embeddings=None, device='cpu',mc_sample_size = 5):

    event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)

    sample_arrival_time = arrival_time[:, 1:]
    sample_event_time = event_time[:, :-1]
    t_last = sample_event_time.max(-1)[0]
    n_batch = sample_arrival_time.size(0)
    n_t = sample_arrival_time.size(1)

    mc_values = torch.rand((n_batch, n_t, mc_sample_size)).to(device) * \
                sample_arrival_time.unsqueeze(-1) + sample_event_time.unsqueeze(-1)

    samples = sample_event_time.unsqueeze(-1).expand((n_batch, n_t, mc_sample_size))
    samples = samples.unsqueeze(1).expand(samples.size(
        0), samples.size(1), samples.size(1), samples.size(-1))

    mc_values_bar = mc_values.unsqueeze(1).expand(mc_values.size(
        0), mc_values.size(1), mc_values.size(1), mc_values.size(-1))

    mc_values_bar = mc_values_bar.transpose(1, 2)
    d = torch.abs((mc_values_bar - samples))


    if kernel.num_types == 1:

        non_event_intensities = kernel(d)
        trigger_integral = non_event_intensities.mean(-1)
        subsequent_mask = get_subsequent_mask(event_type[:, 1:])
        integral = trigger_integral * subsequent_mask
        integral = integral.sum(-1)
        seq_length_mask = (event_type[:, 1:] != 0) * 1
        integral = integral * seq_length_mask
        base_intensity = F.softplus(kernel.base_intensity, beta=1)
        sequence_integral =integral.sum(-1)+base_intensity*t_last
    else:
        sequence_integral = 0
        for i in range(kernel.num_types):
            sample_event_types = event_type[:, 1:]
            sample_embeddings = type_embeddings(sample_event_types)
            xd_bar = sample_embeddings.unsqueeze(1).expand(sample_embeddings.size(
                0), sample_embeddings.size(1), sample_embeddings.size(1), sample_embeddings.size(-1))

            current_embedding = type_embeddings(torch.tensor([i]))
            current_embedding = current_embedding[:, None, None:].expand(xd_bar.size()).transpose(1, 2)
            combined_embeddings = torch.cat([xd_bar, current_embedding], dim=-1)

            non_event_intensities = kernel(d, combined_embeddings, non_event_intensity=True)
            trigger_integral = non_event_intensities.mean(-1)
            subsequent_mask = get_subsequent_mask(sample_event_types)
            integral = trigger_integral * subsequent_mask
            integral = integral.sum(-1)
            seq_length_mask = (event_type[:, 1:] != 0) * 1
            integral = integral * seq_length_mask
            base_intensity = kernel.base_intensity(type_embeddings(torch.tensor([i])))
            sequence_integral += integral.sum(-1) + base_intensity * t_last

    return sequence_integral

In [56]:
sequence_integral

tensor([[ 607.7885,  778.3657,  657.0847,  753.1837,  626.2035,  808.7565,
         1312.2151,  879.8285,  734.3093, 1039.9067,  548.0630,  877.4598,
          775.4640,  579.8536, 1002.8474,  659.2673,  774.5371,  656.8925,
          502.5174,  727.9685,  782.1413,  761.8102,  607.9001,  837.8970,
          619.6664,  630.7327, 1004.1382,  702.7272,  727.8060,  756.7795,
          566.8320,  801.7422,  770.1752,  660.1862,  910.5734,  859.7356,
          631.0574,  579.2618,  614.3885,  768.2284]], grad_fn=<AddBackward0>)

In [49]:
sequence_integral

tensor([[352.7889, 435.0499, 376.0016, 423.2532, 361.4338, 450.1643, 693.3066,
         484.4462, 414.3322, 562.0352, 321.6632, 483.8730, 434.2220, 339.8895,
         544.5463, 378.0758, 434.4902, 375.5476, 301.9268, 411.1165, 437.6788,
         427.7701, 353.0417, 463.2853, 358.7669, 363.9360, 543.5995, 399.6214,
         411.2084, 425.2991, 333.2314, 446.7322, 430.9052, 378.2196, 499.5726,
         474.5798, 364.7369, 337.7893, 354.8994, 430.7442]],
       grad_fn=<AddBackward0>)

In [52]:
sequence_integral

tensor([[254.9995, 343.3158, 281.0830, 329.9305, 264.7697, 358.5922, 618.9084,
         395.3822, 319.9771, 477.8715, 226.3997, 393.5868, 341.2419, 239.9641,
         458.3011, 281.1916, 340.0469, 281.3448, 200.5905, 316.8519, 344.4625,
         334.0402, 254.8584, 374.6116, 260.8994, 266.7967, 460.5387, 303.1058,
         316.5977, 331.4805, 233.6007, 355.0100, 339.2700, 281.9666, 411.0008,
         385.1558, 266.3205, 241.4725, 259.4891, 337.4842]],
       grad_fn=<AddBackward0>)

In [42]:
sequence_integral

tensor([[352.7889, 435.0499, 376.0016, 423.2532, 361.4338, 450.1643, 693.3066,
         484.4462, 414.3322, 562.0352, 321.6632, 483.8730, 434.2220, 339.8895,
         544.5463, 378.0758, 434.4902, 375.5476, 301.9268, 411.1165, 437.6788,
         427.7701, 353.0417, 463.2853, 358.7669, 363.9360, 543.5995, 399.6214,
         411.2084, 425.2991, 333.2314, 446.7322, 430.9052, 378.2196, 499.5726,
         474.5798, 364.7369, 337.7893, 354.8994, 430.7442]],
       grad_fn=<AddBackward0>)

In [39]:
base_intensity*t_last

tensor([[240.8670, 241.3226, 236.6862, 243.9725, 241.0291, 244.3786, 243.3094,
         244.3111, 244.2222, 244.1342, 232.2904, 244.4065, 243.8047, 244.2902,
         243.8539, 242.7897, 244.2795, 238.8226, 243.1743, 243.8973, 244.1301,
         243.9728, 242.5198, 236.6412, 243.8182, 242.9477, 240.3951, 244.2452,
         243.2847, 243.5173, 243.8977, 243.2151, 241.7991, 243.3204, 243.5927,
         241.7718, 244.3802, 237.2125, 238.0700, 244.2785]],
       grad_fn=<MulBackward0>)

In [107]:
kernel.lengthscale(combined_embeddings.unsqueeze(-1)).squeeze(-1).shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (278784000x1 and 64x1)

In [106]:
kernel.lengthscale(combined_embeddings).squeeze(-1)*d

RuntimeError: The size of tensor a (330) must match the size of tensor b (2) at non-singleton dimension 3

In [101]:
d.shape

torch.Size([40, 330, 330, 2])

In [84]:


def get_non_event_intensities(kernel,batch,sample_intensities,device = 'cpu',mc_sample_size = 10):
    
    event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)
    
    sample_arrival_time = arrival_time[:,1:]
    sample_event_time = event_time[:,:-1]
    t_last = sample_arrival_time.max(-1)[0]

    n_batch = sample_arrival_time.size(0)
    n_t = sample_arrival_time.size(1)
    if kernel.num_types==1:
        mc_values = torch.rand((n_batch,n_t,mc_sample_size)).to(device)*sample_arrival_time.unsqueeze(-1)
        base_intensity  = F.softplus(kernel.base_intensity,beta = kernel.betas[-1])
        seq_length_mask = (event_type[:,1:] != 0)*1
        functional_part = kernel(mc_values).mean(-1)*seq_length_mask


    else:
        mc_values = torch.rand((n_batch,n_t,kernel.num_types,mc_sample_size)).to(device)*sample_arrival_time[:,:,None,None]
        functional_part = kernel(mc_values)

# #     constant_part = sample_intensities[:,:-1]*sample_arrival_time    
    
#     seq_length_mask = (event_type[:,1:] != 0)*1
    
    return functional_part


def log_likelihood_loss(sample_intensities, non_event_intensities):
    
    sample_intensities[sample_intensities==0]=1
    log_sum = sample_intensities.log().sum(-1)
    
    return -log_sum + non_event_intensities.sum(-1)



def load_data_loaders(data_path = '../data/simulated/power_hawkes/',batch_size = 40):




    train_data, num_types = load_data(data_path + 'train.pkl', 'train')
    dev_data, _ = load_data(data_path + 'dev.pkl', 'dev')
    test_data, _ = load_data(data_path + 'test.pkl', 'test')

    trainloader = get_dataloader(train_data, batch_size, shuffle=False,t_max =1)
    testloader = get_dataloader(test_data,batch_size, shuffle=False,t_max = 1)
    valloader = get_dataloader(dev_data,batch_size, shuffle=False,t_max =1)


    valid_events = 0
    test_events = 0
    train_events = 0

    for seq in valloader.dataset.event_type:
        valid_events += len(seq)
    for seq in trainloader.dataset.event_type:
        train_events += len(seq)
    for seq in testloader.dataset.event_type:
        test_events  += len(seq)

    return (trainloader,valloader,testloader,train_events,valid_events,test_events)

def train_model(model,trainloader,valloader,testloader,epochs = 250,lr = 0.0001):


    optimizer = optim.Adam(filter(lambda x: x.requires_grad, model.parameters()),
                          lr, betas=(0.9, 0.999), eps=1e-05, weight_decay=0.0)

    for epoch in range(epochs):
        epoch_loss = 0
        for batch in trainloader:
            sample_intensities = get_sample_intensities(model,batch,device = device)
            non_event_intensities = get_non_event_intensities(model,batch,sample_intensities,device = device)
            
            reg_constant =(batch[2]!=0).sum()
            
            
            ll_loss = log_likelihood_loss(sample_intensities,non_event_intensities)
            loss = ll_loss.sum()-model.regularizer_loss()*reg_constant.item()


            epoch_loss+=ll_loss.sum()

            loss.backward()
            optimizer.step()


        epoch_loss = (epoch_loss/train_events).item()

        lengthscale = F.softplus(model.lengthscale,beta = model.betas[0])
        sigma = F.softplus(model.sigma,model.betas[1])
        base_intensity = F.softplus(model.base_intensity,model.betas[2])
        
        print(f'Epoch:{epoch}, NLL:{epoch_loss:.6f}')
        print(f'sigma:{sigma.item():.6f}')
        print(f'lengthscale:{lengthscale.item():.6f}')
        print(f'Base_intensity:{sigma.item():.6f}\n')

        epoch_losses.append(epoch_loss)
        
    return model,epoch_loss

In [86]:
## 2-D
trainloader,valloader,testloader,train_events,valid_events,test_events = load_data_loaders(data_path = '../data/simulated/2_d_hawkes/',batch_size = 40)
for batch in trainloader:
    pass

In [87]:
num_types=2
dtype=32
kernel = magic_kernel(num_types,dtype)
type_emb = nn.Embedding(num_types + 1, dtype, padding_idx=0)


NameError: name 'magic_kernel' is not defined

In [85]:
event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)
embeddings = type_emb(event_type)
sample_intensities = get_sample_intensities(kernel,batch,device= device,embeddings =embeddings)


NameError: name 'batch' is not defined

In [980]:
mc = get_non_event_intensities(kernel,batch,sample_intensities,device = 'cpu',mc_sample_size = 10)

In [1030]:

device = 'cpu'

trainloader,valloader,testloader,train_events,valid_events,test_events = load_data_loaders(data_path = '../data/simulated/2_d_hawkes/',batch_size = 40)
for batch in trainloader:
    event_time, arrival_time, event_type, _ = map(lambda x: x.to(device), batch)
    

num_types = 2
dtype = 2

type_embeddings = nn.Embedding(num_types + 1, dtype, padding_idx=0).to(device)

tensor([[ 9.6656,  9.9690, 10.1388,  ...,  0.0000,  0.0000,  0.0000],
        [ 6.4638,  7.2518,  8.1467,  ...,  0.0000,  0.0000,  0.0000],
        [20.6661, 25.2682, 25.8075,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [59.9563, 72.5169, 73.3082,  ...,  0.0000,  0.0000,  0.0000],
        [11.3614, 12.3198, 14.2027,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.9428, 17.4422, 21.1532,  ...,  0.0000,  0.0000,  0.0000]])

In [1032]:
kernel

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [882]:
subsequent_mask=get_subsequent_mask(event_type)
scores = scores.masked_fill_(subsequent_mask == 0, value=0).sum(-1)
scores_0 = torch.diagonal(scores_0,dim1=1, dim2=-1)

In [993]:
# 1-D

In [1006]:
trainloader,valloader,testloader,train_events,valid_events,test_events = load_data_loaders(data_path = '../data/simulated/power_hawkes/',batch_size = 40)
for batch in trainloader:
    pass

device = 'cpu'

model = magic_kernel(betas = beta_params,num_types=1).to(device)


In [1016]:
event_type

tensor([[2, 1, 1,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 2, 2,  ..., 0, 0, 0],
        ...,
        [2, 2, 2,  ..., 0, 0, 0],
        [2, 1, 1,  ..., 0, 0, 0],
        [1, 2, 2,  ..., 0, 0, 0]])

In [1007]:
sample_intensities = get_sample_intensities(model,batch,device= device)

In [1012]:
mc_val = get_non_event_intensities(model,batch,sample_intensities,device = 'cpu',mc_sample_size = 10)

In [1013]:
mc_val

tensor([[0.6778, 0.7117, 0.1134,  ..., 0.0000, 0.0000, 0.0000],
        [0.6129, 0.5165, 0.6664,  ..., 0.0000, 0.0000, 0.0000],
        [0.1852, 0.6690, 0.6931,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0696, 0.5887, 0.6811,  ..., 0.0000, 0.0000, 0.0000],
        [0.5599, 0.3891, 0.0475,  ..., 0.0000, 0.0000, 0.0000],
        [0.0821, 0.3071, 0.5684,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)

In [650]:
pair_wise_embeddings[1].squeeze(-1)[0][1][0]

tensor(-1.0935, device='cuda:0', grad_fn=<SelectBackward>)

In [762]:
torch.manual_seed(42)
epoch_losses = []
trainloader,valloader,testloader,train_events,valid_events,test_events = load_data_loaders(data_path = '../data/simulated/power_hawkes/',batch_size = 40)

device = 'cuda'


for lr in [0.0001,0.0005]:
    for beta_params in [[1,1,1],[1,1,10],[1,1,5]]:
        model = rational_quadratic_kernel(betas = beta_params,num_types=1).to(device)
        model,loss = train_model(model,trainloader,valloader,testloader,epochs = 250,lr = lr)
        lengthscale = F.softplus(model.lengthscale,beta = model.betas[0])
        sigma = F.softplus(model.sigma,model.betas[1])
        base_intensity = F.softplus(model.base_intensity,model.betas[2])

        print(f'lr:{lr}, betas:{beta_params}, loss:{loss}')
        print(f'sigma:{sigma.item():.6f}')
        print(f'lengthscale:{lengthscale.item():.6f}')
        print(f'Base_intensity:{base_intensity.item():.6f}\n')


RuntimeError: The size of tensor a (99) must match the size of tensor b (40) at non-singleton dimension 1

In [666]:
concat[0][2][2]

tensor([-1.0935, -1.0935], device='cuda:0', grad_fn=<SelectBackward>)

In [551]:
embeddings

tensor([ 2.2181,  0.5232,  0.3466, -0.1973, -1.0546,  1.2780,  0.1453,  0.2311,
         0.0566,  0.4263,  0.5750, -0.6417, -2.2064, -0.7508,  2.8140,  0.3598,
        -1.3407, -0.5854,  0.5362,  0.5246,  1.1412,  0.0516,  0.7281, -0.7106,
        -1.0495,  0.6039, -1.7223, -0.8278,  1.3347,  0.4835, -0.1976,  1.2683],
       device='cuda:0', grad_fn=<SelectBackward>)

In [547]:
type_embeddings(event_type)[0][0]

tensor([ 2.2181,  0.5232,  0.3466, -0.1973, -1.0546,  1.2780,  0.1453,  0.2311,
         0.0566,  0.4263,  0.5750, -0.6417, -2.2064, -0.7508,  2.8140,  0.3598,
        -1.3407, -0.5854,  0.5362,  0.5246,  1.1412,  0.0516,  0.7281, -0.7106,
        -1.0495,  0.6039, -1.7223, -0.8278,  1.3347,  0.4835, -0.1976,  1.2683],
       device='cuda:0', grad_fn=<SelectBackward>)

In [ ]:
t = torch.arange(0,200,0.1)

values = model(t)
values = (values - values.min())/(values.max()-values.min())
real_values = power_law_kernel(t)
real_values = (real_values - real_values.min())/(real_values.max()-real_values.min())

fig, axes = plt.subplots(1, 3,figsize=(17,4))


for ax in axes:
    ax.plot(t,values.detach(),label='Predicted')
    ax.plot(t,real_values,label='Real')
    ax.legend()
    
axes[0].set_xlim(0, 25)
axes[1].set_xlim(0, 10)
axes[2].set_xlim(0, 5)
torch.abs(values -real_values).sum()

In [457]:
torch.clamp(model.sigma,min=0.5, max=1.5)

tensor([0.5000], device='cuda:0', grad_fn=<ClampBackward1>)

In [ ]:
model.sigma